In [1]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
sess = tf.InteractiveSession()
import os
import numpy as np

In [2]:
# Define functions

def conv2d(x, W, dy ,dx):
  return tf.nn.conv2d(x, W, strides=[1, dx, dy, 1], padding='SAME')

def max_pool_axb(x,a,b):
  return tf.nn.max_pool(x, ksize=[1, a, b, 1],
                        strides=[1, a, b, 1], padding='SAME')

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [3]:
# Define the parameters of the network

#-----------ACC 0.774 *** score 81----------------
nx_conv1=16      # filter_width (frequency)
my_conv1=4   # filter_height (time)
dnx_conv1=4    # stride_width
dmy_conv1=1    # stride_height

nx_pool1=4      # pooling width
my_pool1=1      # pooling height

channels_conv1=32    # channels num

nx_conv2=2
my_conv2=16
dnx_conv2=1
dmy_conv2=8

nx_pool2=2
my_pool2=1


channels_conv2=16

size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
num_neurons_fc1=64

print(size_input_fc1)

1024


In [19]:
# Define the parameters of the network



#-----------ACC 0.774 *** score 81----------------
# nx_conv1=16      # filter_width (frequency)
# my_conv1=4   # filter_height (time)
# dnx_conv1=4    # stride_width
# dmy_conv1=1    # stride_height

# nx_pool1=4      # pooling width
# my_pool1=1      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=2
# my_conv2=16
# dnx_conv2=1
# dmy_conv2=8

# nx_pool2=2
# my_pool2=1


# channels_conv2=16

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=64

# print(size_input_fc1)

#-----------ACC 0.65 *** score 74----------------
# nx_conv1=16      # filter_width
# my_conv1=4   # filter_height
# dnx_conv1=4    # stride_width
# dmy_conv1=1    # stride_height

# nx_pool1=4      # pooling width
# my_pool1=4      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=2
# my_conv2=16
# dnx_conv2=1
# dmy_conv2=8

# nx_pool2=2
# my_pool2=2


# channels_conv2=4

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=32

# print(size_input_fc1)

#--------------ACC 0.64  acore 69--------------
# nx_conv1=8      # filter_width
# my_conv1=8  # filter_height
# dnx_conv1=2    # stride_width
# dmy_conv1=2    # stride_height

# nx_pool1=2      # pooling width
# my_pool1=2      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=4
# my_conv2=4
# dnx_conv2=2
# dmy_conv2=2

# nx_pool2=2
# my_pool2=2


# channels_conv2=4

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=32

# print(size_input_fc1)



In [13]:
# Read osu! data

dataList=os.listdir('D:/OSU/SongMat/Train/')
inputOsu=np.empty([0,128*128]);
outputOsu=np.empty([0,4]);
for i in range(np.size(dataList)):
    data=sio.loadmat('D:/OSU/SongMat/Train/'+dataList[i])
    inputOsu=np.append(inputOsu,data['input1'].reshape(-1,128*128),axis=0)
    outputOsu=np.append(outputOsu,data['target1'].reshape(-1,4),axis=0)



In [14]:
outputOsu2=np.zeros([outputOsu.shape[0],2]);
outputOsu2[:,0]=outputOsu[:,0]+outputOsu[:,1]+outputOsu[:,2]
outputOsu2[:,1]=outputOsu[:,3]

In [15]:
x = tf.placeholder("float", shape=[None, 128*128])
y_ = tf.placeholder("float", shape=[None, 2])


x_image = tf.reshape(x, [-1,128,128,1])

W_conv1 = weight_variable([nx_conv1,my_conv1,1,channels_conv1])
b_conv1 = bias_variable([channels_conv1])


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, dnx_conv1, dmy_conv1 ) + b_conv1)
h_pool1 = max_pool_axb(h_conv1, nx_pool1, my_pool1)


W_conv2 = weight_variable([nx_conv2, my_conv2, channels_conv1, channels_conv2])
b_conv2 = bias_variable([channels_conv2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, dnx_conv2, dmy_conv2 ) + b_conv2)
h_pool2 = max_pool_axb(h_conv2, nx_pool2, my_pool2)



W_fc1 = weight_variable([size_input_fc1, num_neurons_fc1])
b_fc1 = bias_variable([num_neurons_fc1])

h_pool2_flat = tf.reshape(h_pool2, [-1, size_input_fc1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


W_fc2 = weight_variable([num_neurons_fc1, 2])
b_fc2 = bias_variable([2])



keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [106]:
#sess.run(tf.initialize_all_variables())  


In [16]:


# merged = tf.summary.merge_all()
# train_writer = tf.summary.FileWriter('/train',sess.graph)
# test_writer = tf.summary.FileWriter('/test')
# tf.global_variables_initializer().run()

sess.run(tf.global_variables_initializer())


In [17]:
saver = tf.train.Saver()


In [18]:
step_size=200
N=inputOsu.shape[0]
print(N)
#shuffle the order
order = np.arange(N-3000)
np.random.shuffle(order)

for n in range(10):
    for i in range(N-3000):
        if i%step_size == 0:
            train_accuracy = accuracy.eval(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 1.0})
#             print("step %d, training accuracy %g" %(i, train_accuracy))
            train_step.run(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 0.5})

    print("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsu[-500:,:], y_: outputOsu2[-500:,:], keep_prob: 1.0}))
    


21944
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652
test accuracy 0.652


In [110]:
save_path = saver.save(sess, "D:/OSU/results/save/",global_step=n)

In [111]:
print(y_conv.eval(feed_dict={x: inputOsu[200:210:1,:], keep_prob: 1.0}))
print ("test accuracy %g"%accuracy.eval(feed_dict={
  x: inputOsu[-500:,:], y_: outputOsu2[-500:,:], keep_prob: 1.0}))

[[ 0.37698266  0.62301731]
 [ 0.35583156  0.6441685 ]
 [ 0.40810573  0.59189427]
 [ 0.38423052  0.61576951]
 [ 0.34308535  0.65691465]
 [ 0.32245824  0.67754179]
 [ 0.38818049  0.61181951]
 [ 0.45136857  0.54863143]
 [ 0.23114154  0.76885849]
 [ 0.43091747  0.5690825 ]]
test accuracy 0.79


In [112]:
# Save the output of new maps 

dataList=os.listdir('D:/OSU/SongMat/Create/')
# inputOsuNew=np.empty([0,128*128]);
for i in range(np.size(dataList)):
    if not dataList[i][0]=='y':
        data=sio.loadmat('D:/OSU/SongMat/Create/'+dataList[i])
        print(data['osuFolder'])
        
        inputOsuNew=data['input'].reshape(-1,128*128)
        outputOsuNew=data['target']
        outputOsu2New=np.zeros([outputOsuNew.shape[0],2]);
        outputOsu2New[:,0]=outputOsuNew[:,0]+outputOsuNew[:,1]+outputOsuNew[:,2]
        outputOsu2New[:,1]=outputOsuNew[:,3]
        
        y_result=np.empty([0,2])
        
        step_size=500
        cursor=0
        print ("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsuNew[-500:,:], y_: outputOsu2New[-500:,:], keep_prob: 1.0}))
        while cursor<inputOsuNew.shape[0]:
            if inputOsuNew[cursor:,:].shape[0]>step_size:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            else:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            cursor+=step_size
            
        data_save={'y2':y_result}
        sio.savemat('D:/OSU/SongMat/Create/y2_'+dataList[i],data_save)


[ 'D:\\Program Files (x86)\\osu!\\Songs\\342826 Maon Kurosaki - Setsuna no Kajitsu\\']
test accuracy 0.51
[ 'D:\\Program Files (x86)\\osu!\\Songs\\342884 Kajiura Yuki - Sis puella magica!\\']
test accuracy 0.620098
['D:\\Program Files (x86)\\osu!\\Songs\\342893 SisterS - Hidamari Days\\']
test accuracy 0.776
[ 'D:\\Program Files (x86)\\osu!\\Songs\\342896 supercell - Kimi no Shiranai Monogatari -TV Edit-\\']
test accuracy 0.722
['D:\\Program Files (x86)\\osu!\\Songs\\342899 SisterS - Hidamari Days\\']
test accuracy 0.9
['D:\\Program Files (x86)\\osu!\\Songs\\343020 fhana - Wonder Stella\\']
test accuracy 0.88
['D:\\Program Files (x86)\\osu!\\Songs\\343258 KOTOKO - INFINITE SKY\\']
test accuracy 0.768
['D:\\Program Files (x86)\\osu!\\Songs\\343365 Hanatan - Ai no Scenario\\']
test accuracy 0.684
['D:\\Program Files (x86)\\osu!\\Songs\\343440 Gain - Bloom\\']
test accuracy 0.706
[ 'D:\\Program Files (x86)\\osu!\\Songs\\343454 Trevor Something - Summer Love\\']
test accuracy 0.736
[ 'D:\\

In [ ]:
data_save={'y':y_result}
sio.savemat('D:/OSU/results/innocence/y_innocence.mat',data_save)

In [ ]:
checkpoint_dir='D:/OSU/results/innocence/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
saver2 = tf.train.Saver()

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver2.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess_tmp = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess_tmp.run(c))

In [ ]:
data_test=sio.loadmat('/Users/dongqihan/Desktop/FrOstNova/granat.mat')
inputOsu_test=data_test['input'].reshape(-1,128*128)
y_result_test=y_conv.eval(feed_dict={x: inputOsu_test, keep_prob: 1.0})
data={'y':y_result_test}
sio.savemat('/Users/dongqihan/Desktop/FrOstNova/y_memory2granat.mat',data)